# Subway status checking

#### Reference
1. MTA Developer Resources (http://web.mta.info/developers/)
2. Google Transit API (https://developers.google.com/transit/gtfs-realtime/examples/python-sample)
3. http://web.mta.info/developers/data/nyct/subway/Stations.csv

In [43]:
# import required package
from google.transit import gtfs_realtime_pb2
import requests
import datetime

In [28]:
# columbia university station id (found from ref 3)
Station = 'Columbia University'
North = '117N'
South = '117S'

In [29]:
# initiate gtfs
feed = gtfs_realtime_pb2.FeedMessage()

In [30]:
# set up url, register an account at REF 1 to get MTA API KEY 
URL = 'http://datamine.mta.info/mta_esi.php?key=MTA_API_KEY>'

In [31]:
# get response
response = requests.get(URL)

In [32]:
# parsing
_ = feed.ParseFromString(response.content)

In [41]:
# check what's inside
#feed.entity[0]

In [71]:
# time
print(datetime.datetime.now())
now = datetime.datetime.now().timestamp()
tol = 10 # minute

2019-04-27 12:31:34.998428


In [70]:
print('== Columbia University Station Waiting Time ==')
for entity in feed.entity:
    # check TripUpdate message only
    if entity.HasField('trip_update'): 
        # check 1 train only
        if entity.trip_update.trip.route_id == '1':
            # extract expected stop time and calculate waiting time
            for msg in entity.trip_update.stop_time_update:
                if (msg.stop_id == North) or (msg.stop_id == South):
                    waiting_time = (msg.arrival.time - now) // 60 
                    if waiting_time < tol and waiting_time >= 0:
                        direction = 'Northbound' if msg.stop_id == North else 'Southbound'
                        print(direction  + ' expected waiting time: ', waiting_time,' mins.')
                        
    if entity.HasField('alert'):
        print('== All Service Alerts ==')
        for alert in entity.alert.informed_entity:
            print( "DELAY: Train", alert.trip.route_id)

== Columbia University Station Waiting Time ==
Northbound expected waiting time:  0.0  mins.
Northbound expected waiting time:  5.0  mins.
Southbound expected waiting time:  4.0  mins.
== All Service Alerts ==
DELAY: Train 1
DELAY: Train 1
DELAY: Train 5
